# Clone Repo

In [1]:
!git clone https://github.com/tommy19970714/fairseq_docker.git

Cloning into 'fairseq_docker'...
Username for 'https://github.com/tommy19970714/fairseq_docker.git': ^C


In [1]:
%cd fairseq_docker/containers

/home/ec2-user/SageMaker/fairseq_docker/container


# Wandb setup

1. Set WANDB_API_KEY in line 72 of `Dockerfile`.
2. And set wandb project name of `wandb_project` in `wav2vec2_base_librispeech.yaml`

# Upload docker to yor ECS

Before upload docker, you have to setup aws cli.
Please check here: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

After you install aws cli, run `aws configure`.
The region of the ecs where the docker will be uploaded must be the same region as the bucket where the dataset will be prepared.

You can upload docker to run `build_and_push.sh`.
The first parameter of shellscript is docker image name.

In [3]:
!sh build_and_push.sh wav2vec2-pretrain

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  14.85kB
Step 1/47 : FROM nvidia/cuda:10.2-cudnn7-devel-ubuntu18.04
10.2-cudnn7-devel-ubuntu18.04: Pulling from nvidia/cuda

2a97ff99: Pulling fs layer 
a9d27eb8: Pulling fs layer 
9583700a: Pulling fs layer 
9bd7f200: Pulling fs layer 
0efc3d4b: Pulling fs layer 
e41ffeef: Pulling fs layer 
8640dce0: Pulling fs layer 
4c3c9304: Pulling fs layer 
Digest: sha256:da5620864a4a4889e3d9ed20a0767ddf3e0bc591ce36364851b36fd4a7ad929bDownloading  567.2MB/838.7MB
Status: Downloaded newer image for nvidia/cuda:10.2-cudnn7-devel-ubuntu18.04
 ---> e37754c25fbb
Step 2/47 : ARG PYTHON_VERSION=3.8
 ---> Running in d1cbbd360cdb
Removing intermediate container d1cbbd360cdb
 ---> ee1e00a288c8
Step 3/47 : RUN apt-get update && apt

# Define IAM role

In [174]:
from sagemaker import get_execution_role

role = get_execution_role()

# Datset

For example, we will have an s3 bucket with the following structure There is no specification for naming the folders or wav files.

```
s3_backet
└── 960h
     ├── xxxx.wav
     ├── xxxx.wav
     ....
```

Define the path of the s3 bucket you prepared.

In [241]:
data_location = 's3://wav2vec-pretrain-datasets/960h'

# Create the session

In [242]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()
sess.default_bucket()

# Create an estimator and fit the model

In [244]:
import boto3

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/wav2vec2-pretrain:latest'.format(account, region)

# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
model = sage.estimator.Estimator(image,
                       role, 1, 'ml.p3.16xlarge',#ml.g4dn.xlarge or p4d.24xlarge or ml.p3.2xlarge
                       volume_size=1000,
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       checkpoint_s3_uri="s3://{}/checkpoints".format(sess.default_bucket()),
                       checkpoint_local_path="/opt/ml/checkpoints/",
                       use_spot_instances=True,
                       max_run=320000,
                       max_wait=400000,
                       sagemaker_session=sess)

In [ ]:
model.fit(data_location)

2021-03-19 19:53:12 Starting - Starting the training job...